# Learned probe direction projections

In [ ]:
import probe_gen.probes as probes
from sklearn.metrics import classification_report
from probe_gen.config import ConfigDict
import torch

# Load the best hyperparameters or set your own
dataset_name_1 = "lists_llama_3b_5k"
dataset_name_2 = "lists_shakespeare_llama_3b_4k"

#cfg = ConfigDict.from_json(probe_type, f"{dataset_name_1}+{dataset_name_2}")
cfg = ConfigDict(layer=12, use_bias=True, normalize=True, C=0.01)

# Create train, val, and test datasets
activations_tensor_1, attention_mask_1, labels_tensor_1 = probes.load_hf_activations_and_labels_at_layer(dataset_name_1, layer=cfg.layer, verbose=True)
activations_tensor_1 = probes.MeanAggregation()(activations_tensor_1, attention_mask_1)
train_dataset_1, val_dataset_1, _ = probes.create_activation_datasets(activations_tensor_1, labels_tensor_1, splits=[3500, 500, 0], verbose=True)

activations_tensor_2, attention_mask_2, labels_tensor_2 = probes.load_hf_activations_and_labels_at_layer(dataset_name_2, layer=cfg.layer, verbose=True)
activations_tensor_2 = probes.MeanAggregation()(activations_tensor_2, attention_mask_2)
train_dataset_2, val_dataset_2, _ = probes.create_activation_datasets(activations_tensor_2, labels_tensor_2, splits=[3500, 500, 0], verbose=True)

probe_1 = probes.SklearnLogisticProbe(cfg)
probe_1.fit(train_dataset_1, val_dataset_1)

probe_2 = probes.SklearnLogisticProbe(cfg)
probe_2.fit(train_dataset_2, val_dataset_2)


In [ ]:
activations_tensor = torch.cat([activations_tensor_1, activations_tensor_2], dim=0)
labels_tensor = torch.cat([labels_tensor_1, labels_tensor_2], dim=0)
train_dataset_mix, val_dataset_mix, _ = probes.create_activation_datasets(activations_tensor, labels_tensor, splits=[7000, 1000, 0], verbose=True)

probe_mix = probes.SklearnLogisticProbe(cfg)
probe_mix.fit(train_dataset_mix, val_dataset_mix)

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

def convert_normalized_probe_to_raw(w_norm, b_norm, mean, std):
    """
    Convert probe weights learned on normalized data to equivalent weights for raw data.
    
    Args:
        w_norm: weights learned on normalized activations (shape [D])
        b_norm: bias learned on normalized activations (scalar)
        mean: normalization mean vector (shape [D])
        std: normalization std vector (shape [D])
    
    Returns:
        w_raw: equivalent weights for raw activations (shape [D])
        b_raw: equivalent bias for raw activations (scalar)
    """
    w_raw = w_norm / std
    b_raw = b_norm - torch.dot(w_norm / std, mean)
    
    return w_raw, b_raw

# Usage:
w_norm_1 = torch.tensor(probe_1.classifier.coef_.flatten()).to(torch.float32)
b_norm_1 = probe_1.classifier.intercept_[0]
mean_1 = torch.tensor(probe_1.transformation_mean.flatten()).to(torch.float32)
std_1 = torch.tensor(probe_1.transformation_std.flatten()).to(torch.float32)

w_raw_1, b_raw_1 = convert_normalized_probe_to_raw(w_norm_1, b_norm_1, mean_1, std_1)

# Same for probe 2
w_norm_2 = torch.tensor(probe_2.classifier.coef_.flatten()).to(torch.float32)
b_norm_2 = probe_2.classifier.intercept_[0]
mean_2 = torch.tensor(probe_2.transformation_mean.flatten()).to(torch.float32)
std_2 = torch.tensor(probe_2.transformation_std.flatten()).to(torch.float32)

w_raw_2, b_raw_2 = convert_normalized_probe_to_raw(w_norm_2, b_norm_2, mean_2, std_2)

# Same for probe mix
w_norm_mix = torch.tensor(probe_mix.classifier.coef_.flatten()).to(torch.float32)
b_norm_mix = probe_mix.classifier.intercept_[0]
mean_mix = torch.tensor(probe_mix.transformation_mean.flatten()).to(torch.float32)
std_mix = torch.tensor(probe_mix.transformation_std.flatten()).to(torch.float32)

w_raw_mix, b_raw_mix = convert_normalized_probe_to_raw(w_norm_mix, b_norm_mix, mean_mix, std_mix)

print(w_raw_mix.shape)
a = torch.dot(F.normalize(w_raw_mix, dim=0), F.normalize(w_raw_1, dim=0))
b = torch.dot(F.normalize(w_raw_mix, dim=0), F.normalize(w_raw_2, dim=0))

# Rest of your plotting code...
positive_activations_1 = activations_tensor_1[labels_tensor_1 == 1]
negative_activations_1 = activations_tensor_1[labels_tensor_1 == 0]
positive_activations_2 = activations_tensor_2[labels_tensor_2 == 1]
negative_activations_2 = activations_tensor_2[labels_tensor_2 == 0]

plt.figure(figsize=(10, 8))

# Plot scatter points
projections_x = positive_activations_1 @ w_raw_1
projections_y = positive_activations_1 @ w_raw_2
plt.scatter(projections_x, projections_y, label="Multiple choice - Pos", color='red', alpha=0.1)

projections_x = negative_activations_1 @ w_raw_1
projections_y = negative_activations_1 @ w_raw_2
plt.scatter(projections_x, projections_y, label="Multiple choice - Neg", color='blue', alpha=0.1)

projections_x = positive_activations_2 @ w_raw_1
projections_y = positive_activations_2 @ w_raw_2
plt.scatter(projections_x, projections_y, label="Arguments - Pos", color='orange', alpha=0.1)

projections_x = negative_activations_2 @ w_raw_1
projections_y = negative_activations_2 @ w_raw_2
plt.scatter(projections_x, projections_y, label="Arguments - Neg", color='green', alpha=0.1)

# Plot corrected decision boundaries
plt.axvline(x=-b_raw_1, color='red', linestyle='--', linewidth=2, 
            label='Multiple Choice Probe Decision Boundary')
plt.axhline(y=-b_raw_2, color='blue', linestyle='--', linewidth=2, 
            label='Arguments Probe Decision Boundary')



def get_decision_line_points(A, B, C, xlim, ylim):
    """Find intersection points of line A*x + B*y + C = 0 with plot boundaries"""
    points = []
    
    # Check intersections with x boundaries
    for x_bound in xlim:
        if abs(B) > 1e-10:
            y = -(A * x_bound + C) / B
            if ylim[0] <= y <= ylim[1]:
                points.append((x_bound, y))
    
    # Check intersections with y boundaries  
    for y_bound in ylim:
        if abs(A) > 1e-10:
            x = -(B * y_bound + C) / A
            if xlim[0] <= x <= xlim[1]:
                points.append((x, y_bound))
    
    return points[:2]  # We only need 2 points to draw a line

A = torch.dot(w_raw_mix, direction_1_normalized)  # coefficient of α (x-axis)
B = torch.dot(w_raw_mix, direction_2_normalized)  # coefficient of β (y-axis)
C = b_raw_mix

# Get current plot limits to draw line across entire plot
xlim = plt.xlim()
ylim = plt.ylim()

points = get_decision_line_points(A, B, C, xlim, ylim)

plt.plot([points[0][0], points[1][0]], [points[0][1], points[1][1]], color='green', linestyle='--', linewidth=2,
                label='Both Probe Decision Boundary')


plt.xlabel('Projection onto Multiple Choice Probe')
plt.ylabel('Projection onto Arguments Probe')
legend = plt.legend()
for handle in legend.legend_handles:
    handle.set_alpha(1.0)
plt.title('Probe Directions and Decision Boundaries (Sycophancy)')
plt.grid(True, alpha=0.3)
plt.show()

# Does use_bias change anything?

In [ ]:
import probe_gen.probes as probes
from sklearn.metrics import classification_report
from probe_gen.config import ConfigDict
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the best hyperparameters or set your own
dataset_names = [
    "sycophancy_ministral_8b_4k",
]
cosine_similarities = []


for dataset_name in dataset_names:

    # Create train, val, and test datasets
    activations_tensor, attention_mask, labels_tensor = probes.load_hf_activations_and_labels_at_layer(dataset_name, 12, verbose=True)
    activations_tensor = probes.MeanAggregation()(activations_tensor, attention_mask)
    train_dataset, _, _ = probes.create_activation_datasets(activations_tensor, labels_tensor, splits=[3500, 0, 0], verbose=True)

    cfg_1 = ConfigDict(layer=12, use_bias=True, normalize=True, C=0.01)
    probe_1 = probes.SklearnLogisticProbe(cfg_1)
    probe_1.fit(train_dataset, None)

    cfg_2 = ConfigDict(layer=12, use_bias=False, normalize=True, C=0.01)
    probe_2 = probes.SklearnLogisticProbe(cfg_2)
    probe_2.fit(train_dataset, None)

    weights_1 = probe_1.classifier.coef_.flatten()
    weights_2 = probe_2.classifier.coef_.flatten() 

    sim = cosine_similarity([weights_1], [weights_2])[0, 0]
    print(f"{dataset_name}: {sim}")
    cosine_similarities.append(sim.item())

print(cosine_similarities)

In [4]:
array = []
for x in cosine_similarities:
    array.append(x.item())
print(array)

[0.9921962411168163, 0.9909800209336572, 0.9938872232115757, 0.9826005679340069, 0.9888519132159277, 0.9893070271046311]


[0.9921962411168163, 0.9909800209336572, 0.9938872232115757, 0.9826005679340069, 0.9888519132159277, 0.9893070271046311, 0.9956215483248319, 0.9990389554915939]

# Dataset size test

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
import probe_gen.probes as probes

train_dataset_names = ['science_llama_3b_5k', 'science_llama_3b_prompted_5k', 'science_qwen_3b_5k']
test_dataset_name = 'science_llama_3b_1k'

# Get train activations and labels
train_activations_tensors = []
train_labels_tensors = []
for i in range(len(train_dataset_names)):
    activations_tensor, attention_mask, labels_tensor = probes.load_hf_activations_and_labels_at_layer(train_dataset_names[i], 12, verbose=True)
    activations_tensor = probes.MeanAggregation()(activations_tensor, attention_mask)
    train_activations_tensors.append(activations_tensor)
    train_labels_tensors.append(labels_tensor)

# Create test dataset
activations_tensor, attention_mask, labels_tensor = probes.load_hf_activations_and_labels_at_layer(test_dataset_name, 12, verbose=True)
activations_tensor = probes.MeanAggregation()(activations_tensor, attention_mask)
_, _, test_dataset = probes.create_activation_datasets(activations_tensor, labels_tensor, splits=[0, 0, 1000], verbose=True)

In [ ]:
from probe_gen.config import ConfigDict

train_sizes = [i for i in range(500, 5000, 200)]

rocs = [[] for _ in range(len(train_dataset_names))]

for train_size in train_sizes:
    print(f"##### Training with size {train_size} #####")
    # Create train, val, and test datasets
    for i in range(len(train_dataset_names)):
        train_dataset, _, _ = probes.create_activation_datasets(train_activations_tensors[i], train_labels_tensors[i], splits=[train_size, 0, 0], verbose=False)
        probe = probes.SklearnLogisticProbe(ConfigDict(use_bias=True, C=0.01, normalize=True))
        probe.fit(train_dataset, None)
        eval_dict, _, _ = probe.eval(test_dataset)
        rocs[i].append(eval_dict['roc_auc'])
        

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Plot each line with different styles and labels
plt.plot(train_sizes, rocs[0], label='On', marker='o', linewidth=2)
plt.plot(train_sizes, rocs[1], label='Prompted', marker='s', linewidth=2)
plt.plot(train_sizes, rocs[2], label='Off', marker='^', linewidth=2)

# Customize the plot
plt.title('Effect of train size on generalization to on policy (science)')
plt.xlabel('Train size')
plt.ylabel('roc_auc')
plt.legend()
plt.grid(True, alpha=0.3)

# Optional: set y-axis limits if needed
# plt.ylim(70, 100)

# Display the plot
plt.tight_layout()
plt.show()

# Confusion Matrices

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
import probe_gen.probes as probes
from sklearn.metrics import classification_report, confusion_matrix
from probe_gen.config import ConfigDict

activations_tensor, attention_mask, labels_tensor = probes.load_hf_activations_and_labels_at_layer('sycophancy_llama_3b_4k', 12, verbose=True)
activations_tensor = probes.MeanAggregation()(activations_tensor, attention_mask)
# Create train, val, and test datasets
train_dataset, val_dataset, _ = probes.create_activation_datasets(activations_tensor, labels_tensor, splits=[3500, 500, 0], verbose=True)

activations_tensor, attention_mask, labels_tensor = probes.load_hf_activations_and_labels_at_layer('sycophancy_arguments_llama_3b_4k', 12, verbose=True)
activations_tensor = probes.MeanAggregation()(activations_tensor, attention_mask)
# Create train, val, and test datasets
_, _, test_dataset = probes.create_activation_datasets(activations_tensor, labels_tensor, splits=[0, 0, 1000], verbose=True)

In [ ]:
import matplotlib.pyplot as plt

probe = probes.SklearnLogisticProbe(ConfigDict(use_bias=True, C=0.01, normalize=True))
probe.fit(train_dataset, val_dataset)
eval_dict, y_pred, y_pred_proba = probe.eval(test_dataset)

cm = confusion_matrix(test_dataset['y'], y_pred)

fig, ax = plt.subplots()
im = ax.imshow(cm, interpolation='nearest', cmap="Reds")
ax.figure.colorbar(im, ax=ax)

# Set tick marks
class_names = np.arange(cm.shape[0])
ax.set(
    xticks=np.arange(cm.shape[1]),
    yticks=np.arange(cm.shape[0]),
    xticklabels=class_names,
    yticklabels=class_names,
    ylabel="True label",
    xlabel="Predicted label"
)

# Rotate x-axis labels
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Add text labels
fmt = "d"
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], fmt),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")

fig.tight_layout()
plt.show()

# Significance of small roc_aucs   

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
import probe_gen.probes as probes
from sklearn.metrics import classification_report
from probe_gen.config import ConfigDict


activations_tensor, attention_mask, labels_tensor = probes.load_hf_activations_and_labels_at_layer('sycophancy_llama_3b_4k', 12, verbose=True)
activations_tensor = probes.MeanAggregation()(activations_tensor, attention_mask)
# Create train, val, and test datasets
train_dataset, val_dataset, _ = probes.create_activation_datasets(activations_tensor, labels_tensor, splits=[3500, 500, 0], verbose=True)

probe = probes.SklearnLogisticProbe(ConfigDict(use_bias=True, C=0.01, normalize=True))
probe.fit(train_dataset, val_dataset)

In [ ]:
activations_tensor, attention_mask, labels_tensor = probes.load_hf_activations_and_labels_at_layer('sycophancy_arguments_llama_3b_4k', 12, verbose=True)
activations_tensor = probes.MeanAggregation()(activations_tensor, attention_mask)
# Create train, val, and test datasets
_, _, test_dataset = probes.create_activation_datasets(activations_tensor, labels_tensor, splits=[0, 0, 1000], verbose=True)

In [ ]:

def random_unit_vector(n: int) -> np.ndarray:
    v = np.random.normal(size=(1, n))   # Gaussian sample
    v /= np.linalg.norm(v)              # Normalize to unit length
    return v

rand_probe = probes.SklearnLogisticProbe(ConfigDict(use_bias=True, C=0.01, normalize=True))
rand_probe.fit(train_dataset, val_dataset)

roc_aucs = []

for i in range(1000):
    # Set random probe direction
    rand_probe.classifier.coef_ = random_unit_vector(probe.classifier.coef_.shape[1])

    # Check if cosine similarity is near orthogonal
    weights_1 = probe.classifier.coef_.flatten()
    weights_2 = rand_probe.classifier.coef_.flatten()
    if abs(cosine_similarity([weights_1], [weights_2])[0, 0]) < 0.018:
        eval_roc_auc = rand_probe.eval(test_dataset)[0]['roc_auc']
        roc_aucs.append(eval_roc_auc)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(roc_aucs, bins=25, edgecolor='black')  # adjust bins as needed
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Histogram of roc_aucs for near orthogonal vectors")
plt.show()